## Flexural Subsidence

This example explores how to use a BMI implementation using sedflux's subsidence model as an example.

### Links
* [sedflux source code](https://github.com/mcflugen/sedflux): Look at the files that have *deltas* in their name.
* [sedflux description on CSDMS](https://csdms.colorado.edu/wiki/Model_help:Sedflux): Detailed information on the CEM model.

### Interacting with the Subside BMI using Python

Some magic that allows us to view images within the notebook.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

Import the `Subside` class, and instantiate it. In Python, a model with a BMI will have no arguments for its constructor. Note that although the class has been instantiated, it's not yet ready to be run. We'll get to that later!

In [ ]:
import pymt.models
subside = pymt.models.Subside()

Even though we can't run our subsidence model yet, we can still get some information about it. *Just don't try to run it.* Some things we can do with our model are get the names of the input variables.

In [ ]:
subside.output_var_names

In [ ]:
subside.input_var_names

We can also get information about specific variables. Here we'll look at some info about lithospheric deflections. This is the main input of the Subside model. Notice that BMI components always use [CSDMS standard names](http://csdms.colorado.edu/wiki/CSDMS_Standard_Names). With that name we can get information about that variable and the grid that it is on.

OK. We're finally ready to run the model. Well not quite. First we initialize the model with the BMI **initialize** method. Normally we would pass it a string that represents the name of an input file. For this example we'll pass **None**, which tells Cem to use some defaults.

In [ ]:
config_file, config_folder = subside.setup()

In [ ]:
subside.initialize(config_file, dir=config_folder)

In [ ]:
subside.var["earth_material_load__pressure"]

In [ ]:
subside.grid[0].node_shape

Before running the model, let's set an input parameter - the overlying load.

In [ ]:
load = np.zeros((500, 500))
load[250, 250] = 1e3

The main output variable for this model is *deflection*. In this case, the CSDMS Standard Name is:

    "lithosphere__increment_of_elevation"

First we find out which of Subside's grids contains deflection. 

In [ ]:
subside.var['lithosphere__increment_of_elevation']

With the *grid_id*, we can now get information about the grid. For instance, the number of dimension and the type of grid (structured, unstructured, etc.). This grid happens to be *uniform rectilinear*. If you were to look at the "grid" types for wave height and period, you would see that they aren't on grids at all but instead are scalars.

In [ ]:
subside.grid[0]

Because this grid is uniform rectilinear, it is described by a set of BMI methods that are only available for grids of this type. These methods include:
* get_grid_shape
* get_grid_spacing
* get_grid_origin

Allocate memory for the water depth grid and get the current values from `cem`.

In [ ]:
subside.set_value("earth_material_load__pressure", load)

In [ ]:
subside.update()

In [ ]:
dz = subside.get_value('lithosphere__increment_of_elevation')

In [ ]:
plt.imshow(dz.reshape((500, 500)))

In [ ]:
load[125, 125] = 2e3
subside.set_value("earth_material_load__pressure", load)
subside.update()

In [ ]:
dz = subside.get_value('lithosphere__increment_of_elevation')
plt.imshow(dz.reshape((500, 500)))
plt.colorbar()